# LangChain Toolbox + MCP (Restaurant Demo)

This notebook shows how to run a local MCP server and call its tools with the Toolbox LangChain SDK and langchain-openai.


## Setup (uv init + venv)

```bash
uv init
uv venv
.\.venv\Scripts\activate
uv add mcp toolbox-langchain langchain-openai langgraph python-dotenv
```

Create a `.env` file in this folder:

```bash
OPENAI_API_KEY=...
TOOLBOX_URL=http://127.0.0.1:5000
```


## Start the MCP server

Run this in a separate terminal:

```bash
python server.py
```

The server listens on http://127.0.0.1:5000.


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
TOOLBOX_URL = os.getenv("TOOLBOX_URL", "http://127.0.0.1:5000")


## Load tools (toolset)


In [2]:
from toolbox_langchain import ToolboxClient

async def load_tools():
    async with ToolboxClient(TOOLBOX_URL) as toolbox:
        return toolbox.load_toolset()

tools = await load_tools()
[tool.name for tool in tools]


['list_pizza_prices',
 'get_pizza_price',
 'get_opening_hours',
 'get_restaurant_info']

## Use case 1: manual tool calls


In [ ]:
async with ToolboxClient(TOOLBOX_URL) as toolbox:
    list_prices = toolbox.load_tool("list_pizza_prices")
    get_price = toolbox.load_tool("get_pizza_price")
    get_hours = toolbox.load_tool("get_opening_hours")

    prices = list_prices.invoke({})
    margherita = get_price.invoke({"pizza": "margherita"})
    saturday = get_hours.invoke({"day": "sat"})

prices, margherita, saturday


## Use case 2: load a single tool


In [3]:
async def load_single_tool():
    async with ToolboxClient(TOOLBOX_URL) as toolbox:
        get_hours = toolbox.load_tool("get_opening_hours")
        return get_hours.invoke({"day": "all"})

all_hours = await load_single_tool()
all_hours


'{\n  "hours": {\n    "mon": "11:00-22:00",\n    "tue": "11:00-22:00",\n    "wed": "11:00-22:00",\n    "thu": "11:00-22:00",\n    "fri": "11:00-23:00",\n    "sat": "12:00-23:00",\n    "sun": "12:00-21:00"\n  }\n}'

## Use case 3: LangChain + OpenAI (agent via LangGraph)

This uses a small ReAct-style agent that can call the MCP tools automatically.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

async def run_agent():
    async with ToolboxClient(TOOLBOX_URL) as toolbox:
        tools = toolbox.load_toolset()
        agent = create_agent(model, tools)

        prompt = "What are the pizza prices and the opening hours on Saturday?"
        result = agent.invoke({"messages": [("user", prompt)]})
        return result["messages"][-1].content

await run_agent()


ImportError: cannot import name 'create_react_agent' from 'langchain' (c:\Users\User\Desktop\LangChainToolbox\.venv\Lib\site-packages\langchain\__init__.py)

## Use case 4: LangChain + OpenAI (bind_tools)

This binds the MCP tools to the model and runs tool calls manually.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import ToolMessage

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

async def run_with_bind_tools():
    async with ToolboxClient(TOOLBOX_URL) as toolbox:
        tools = toolbox.load_toolset()
        tool_map = {tool.name: tool for tool in tools}
        model_with_tools = model.bind_tools(tools)

        prompt = "What are the pizza prices and the opening hours on Saturday?"
        first = model_with_tools.invoke(prompt)

        if not first.tool_calls:
            return first.content

        tool_messages = []
        for call in first.tool_calls:
            result = tool_map[call["name"]].invoke(call["args"])
            tool_messages.append(
                ToolMessage(content=str(result), tool_call_id=call["id"])
            )

        final = model_with_tools.invoke([("user", prompt), first, *tool_messages])
        return final.content

await run_with_bind_tools()
